# S&P 500  - Kazzaz's Walkthru
## Connect to the SP500 database file
Let's start with importing the libraries we need.

In [1]:
import sqlite3
import pandas as pd
import numpy as np

To keep us all aligned, let's change the "current working directory" (cwd) to the starting point we see in the notebook's File Browser on the left.  The cwd is the default place for our notebook to read/write files.  It will also be the starting point when we reference files.

In [2]:
import os

# changing the "current working directory"
os.chdir('/home/ec2-user/SageMaker/')

# confirming cwd is updated
os.getcwd()

'/home/ec2-user/SageMaker'

Now with the cwd set to `/home/ec2-user/SageMaker`, any file references we make will start with this location.  Let's see it in action.  Next we need to connect to the sqlite database file.  I already cloned our **Resources** repo using the clone button to the left.  If I right click on the sp500 sqlite file and click "copy path", I get the following: `resources/phase3/sp500/sp500.sqlite`.  When I tell the notebook to head to that database fail path, it will start looking from the cwd.  This results in the absolute file to the file being `/home/ec2-user/SageMaker/resources/phase3/sp500/sp500.sqlite`. 

Let's connect to the data base file using the following:

In [3]:
dbSP500 = sqlite3.connect('resources/phase3/sp500/sp500.sqlite')

If you don't see a response here, then you're good to go!

If you do see an error, it means something is wrong with either the cwd or the file path for the database.  Double check your work.

## Dataset Questions
Let's work through a few example questions to get our head around this dataset.

### How large is this dataset?
Depends on what we mean by "large" here.  Are we talking about tables, rows, or columns?  Let's answer everything to play safe.

In [4]:
queryTables = """
select name from sqlite_master
where type = 'table'
"""

dfTables = pd.read_sql_query(queryTables, dbSP500)

print(dfTables)

    name
0  sp500


In [5]:
queryRows = """
select count(*) from sp500
"""

dfRows = pd.read_sql_query(queryRows, dbSP500)

print(dfRows)

   count(*)
0     17378


We used the `count(*)` SQL statement to get a count of records in the table.  Let's go ahead and pop all that data into a Pandas dataframe.

In [6]:
queryData = """
select * from sp500
"""

dfSP500 = pd.read_sql_query(queryData, dbSP500)

dfSP500.head(5) # show the first five records

,id,Date,Open,High,Low,Close,Adj Close,Volume
0,1,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,2,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,3,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,4,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,5,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


Now with the data in a data frame, let's count the number of columns we found.  There's several ways to go about this.  Here we'll use a Python loop and count the number of iterations (i) we go through.

In [7]:
i = 0 # set an iteration variable to zero

for col in dfSP500: # set a loop based on the columns in the dataframe
    print(col) # print the column to screen
    i += 1 # increment i by 1
    
print("\nFound %s columns" % i) # print the column count to screen

id
Date
Open
High
Low
Close
Adj Close
Volume

Found 8 columns


### What is the highest S&P500 price recorded in the dataset?

A simple way to do this is to find the MAX value in the column "high".

In [8]:
# SQL
queryHighPrice = """
select max(high) from sp500
"""

resultHighPriceSQL = pd.read_sql_query(queryHighPrice, dbSP500)

print("SQL Result: \n\n %s" % resultHighPriceSQL)

print("\n\n\n") # print three blank lines

# Pandas
resultHighPricePandas = dfSP500['High'].max()
# NOTE: The source we hit here is the dataframe with all the records

print("Pandas Result: \n\n %s" % resultHighPricePandas)

SQL Result: 

   max(high)
0    999.61




Pandas Result: 

 999.61


While we technically answered the question of how high the price was, we don't know when that happened.  Let's answer the question fully: how much was the highest price and when was it?

In [9]:
# SQL
queryHighPriceWhen = """
select * from sp500
where high = (select max(high) from sp500)
"""

resultHighPriceWhenSQL = pd.read_sql_query(queryHighPriceWhen, dbSP500)

print("SQL Result: \n\n %s" % resultHighPriceWhenSQL)

print("\n\n\n") # print three blank lines

# Pandas

resultHighPriceWhenPandas = dfSP500[dfSP500['High']==dfSP500['High'].max()]

print("Pandas Result: \n\n%s" % resultHighPriceWhenPandas)

SQL Result: 

       id        Date    Open    High     Low   Close Adj Close      Volume
0  15004  2009-08-19  986.88  999.61  980.62  996.46    996.46  4257000000




Pandas Result: 

          id        Date    Open    High     Low   Close Adj Close      Volume
15003  15004  2009-08-19  986.88  999.61  980.62  996.46    996.46  4257000000


### What's the max, average, and min close price for latest 20 years in the S&P500 dataset?

Let's unpack what that's asking us.
* We expect a single row for every year recorded.
* We expect the latest 20 years in the dataset.
* We need three different calculations (max, avg, min) for each year recorded.

In [10]:
#SQL
queryYearlySummaryQuery = """
select 
  substr(date, 1,4) as year,
  max(close) as max_close, 
  avg(close) as avg_close, 
  min(close) as min_close
from sp500
group by substr(date, 1,4)
order by substr(date, 1,4) desc
limit 20
"""

resultYearlySummarySQL = pd.read_sql_query(queryYearlySummaryQuery, dbSP500)

print("SQL Result: \n\n %s" % resultYearlySummarySQL)

SQL Result: 

     year max_close    avg_close min_close
0   2019   2670.71  2593.305294   2447.89
1   2018   2930.75  2746.214422    2351.1
2   2017   2690.16  2449.076375   2257.83
3   2016   2271.72  2094.651270   1829.08
4   2015   2130.82  2061.067738   1867.61
5   2014   2090.57  1931.376111   1741.89
6   2013   1848.36  1643.798968   1457.15
7   2012   1465.77  1379.354160   1277.06
8   2011   1363.61  1267.638810   1099.23
9   2010   1259.78  1139.965516   1022.58
10  2009    998.04   948.046389   1002.63
11  2008    998.01  1220.042055   1003.35
12  2007   1565.15  1477.184343   1374.12
13  2006   1427.09  1310.461235   1223.69
14  2005   1272.74  1207.229444    1137.5
15  2004   1213.55  1130.649444   1063.23
16  2003    999.74   965.227540    1000.3
17  2002    992.72   993.934802   1006.29
18  2001    984.54  1194.178992   1003.45
19  2000   1527.46  1427.221071   1264.74


In [37]:
# Pandas

dfSP500Work = dfSP500

dfSP500Work['Year'] = dfSP500Work['Date'].str.slice(stop=4) # create year column
dfSP500Work['Close'] = pd.to_numeric(dfSP500Work['Close']) # make Close a numeric field

dfSP500Work.groupby(['Year']).agg(
    {'Close': ['max', 'mean', 'min']}
        ).sort_values(by='Year', ascending=False
             ).head(20)


Close                      
          max         mean      min
Year                               
2019  2670.71  2593.305294  2447.89
2018  2930.75  2746.214422  2351.10
2017  2690.16  2449.076375  2257.83
2016  2271.72  2094.651270  1829.08
2015  2130.82  2061.067738  1867.61
2014  2090.57  1931.376111  1741.89
2013  1848.36  1643.798968  1457.15
2012  1465.77  1379.354160  1277.06
2011  1363.61  1267.638810  1099.23
2010  1259.78  1139.965516  1022.58
2009  1127.78   948.046389   676.53
2008  1447.16  1220.042055   752.44
2007  1565.15  1477.184343  1374.12
2006  1427.09  1310.461235  1223.69
2005  1272.74  1207.229444  1137.50
2004  1213.55  1130.649444  1063.23
2003  1111.92   965.227540   800.73
2002  1172.51   993.934802   776.76
2001  1373.73  1194.178992   965.80
2000  1527.46  1427.221071  1264.74